## Import modules

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torch.autograd import Variable
import time
import torchvision.models as models
import os
from torch.utils.data.sampler import SubsetRandomSampler
from torch.autograd import Variable

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV

## Load dataset

In [ ]:
print(1)
from shutil import copyfile
for i in range(10000, 20000):
    copyfile('jsma_perturbed_images/0.1/' + str(i) + '.png', 'safetynet-jsma-test/photos/' + str(i) + '.png')

In [ ]:
print(1)
# CIFAR10 Test dataset and dataloader declaration
transform = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor()])

trainset = torchvision.datasets.ImageFolder(root='safetynet-normal-train', transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=10000,
                                          shuffle=True, num_workers=2)

adversarialset = torchvision.datasets.ImageFolder(root='safetynet-jsma-train', transform=transform)
adversarialloader = torch.utils.data.DataLoader(adversarialset, batch_size=10000,
                                          shuffle=True, num_workers=2)

totaltrainloader = torch.utils.data.DataLoader(
             torch.utils.data.ConcatDataset([
                 trainset,
                 adversarialset]),
             batch_size=20000, shuffle=False,
             num_workers=5)

print(2)

testset = torchvision.datasets.ImageFolder(root='safetynet-normal-test', transform=transform)
testadversarialset = torchvision.datasets.ImageFolder(root='safetynet-jsma-test', transform=transform)
totaltestloader = torch.utils.data.DataLoader(
             torch.utils.data.ConcatDataset([
                 testset,
                 testadversarialset]),
             batch_size=20000, shuffle=False,
             num_workers=5)

print(3)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

## Load model

In [ ]:
# Define what device we are using
print("CUDA Available: ",torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the network
model = models.vgg16(pretrained=True)
model.fc = nn.Linear(2048, 10)

# freeze front layers
ct = 0
for child in model.children():
    ct += 1
    if ct < 5:
        for param in child.parameters():
            param.requires_grad = True
model = model.to(device)
# model.eval()

## Train 

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, gamma=0.1, step_size=30)
criterion = nn.CrossEntropyLoss()

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

In [ ]:
def train(train_loader, model, criterion, optimizer, epoch):
    """
        Run one train epoch
    """
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    total = 0
    correct = 0
    
    # switch to train mode
    model.train()

    end = time.time()
    for i, (input, target) in enumerate(train_loader):

        # measure data loading time
        data_time.update(time.time() - end)

        target = target.to(device)
        input = input.to(device)
        target_var = target

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # measure accuracy and record loss
        _, predicted = output.max(1)
        losses.update(loss.item(), input.size(0))
        total += target.size(0)
        correct += predicted.eq(target).sum().item()


        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 50 == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Accuracy {acc:.3f}'.format(
                      epoch, i, len(train_loader), batch_time=batch_time,
                      data_time=data_time, loss=losses, acc=correct/total))


In [ ]:
def save_checkpoint(state,filename='checkpoint.pth.tar'):
    """
    Save the training model
    """
    torch.save(state, filename)

In [ ]:
global best_prec1
best_prec1 = 0

for epoch in range(50):

    # train for one epoch
    print('current lr {:.5e}'.format(optimizer.param_groups[0]['lr']))
    train(trainloader, model, criterion, optimizer, epoch)
    lr_scheduler.step()

    if epoch > 0 and epoch % 50 == 0:
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict()
        },  filename=os.path.join('./checkpoints/', 'checkpoint2.th'))

    save_checkpoint({
        'state_dict': model.state_dict()
    },  filename=os.path.join('./checkpoints/', 'model2.th'))

## Build the safetynet

In [ ]:
print("START")
model.load_state_dict(torch.load('./checkpoints/model2.th')['state_dict'])
model.eval()
model2 = nn.Sequential(*list(model.children())[:-2])
#print(model2)
inputs, labels = next(iter(totaltrainloader))
inputs, labels = Variable(inputs), Variable(labels)
#outputs = model2(inputs)
#print(inputs)
#print(outputs)
print("DONE")


In [ ]:
nsamples, nx, ny, nz = inputs.shape
print(nsamples, nx, ny, nz)
inputs = inputs.reshape(nsamples,nx*ny*nz)

print(inputs.shape)

In [ ]:

totaltrainsetlabels = []
for i in range(10000):
    totaltrainsetlabels.append(1)
    
for i in range(10000):
    totaltrainsetlabels.append(-1)
    
totaltrainsetlabels = np.array(totaltrainsetlabels)

C_2d_range = [1e-2, 1, 1e2]
gamma_2d_range = [1e-1, 1, 1e1]
classifiers = []
for C in C_2d_range:
    for gamma in gamma_2d_range:
        print(C, gamma)
        clf = SVC(C=C, gamma=gamma)
        clf.fit(inputs.detach().numpy(), totaltrainsetlabels)
        
        classifiers.append((C, gamma, clf))

In [ ]:
it = iter(totaltestloader)

inputs2, labels2 = next(it)
inputs2, labels2 = Variable(inputs2), Variable(labels2)

In [ ]:
nsamples, nx, ny, nz = inputs2.shape
print(nsamples, nx, ny, nz)
inputs2 = inputs2.reshape(nsamples,nx*ny*nz)

print(inputs)
print(inputs2)

In [ ]:
results = []
for classifier in classifiers:
    results.append(classifier[2].predict(inputs2.tolist()))

In [ ]:
TP = 0
FP = 0
TN = 0
FN = 0

for result in results:
    print(result)
    for i in range(10000):
        if result[i] == 1:
            TP += 1
        else:
            FN += 1
    for i in range(10000, 20000):
        if result[i] == -1:
            TN += 1
        else:
            FP += 1
    print(TP)
    print(FP)
    print(TN)
    print(FN)
    print()
    print()
    TP = 0
    FP = 0
    TN = 0
    FN = 0

In [ ]:
temp = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
temploader = torch.utils.data.DataLoader(trainset, batch_size=1,
                                          shuffle=True, num_workers=2)

temp2 = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
temp2loader = torch.utils.data.DataLoader(trainset, batch_size=1,
                                          shuffle=True, num_workers=2)

temptotalloader = torch.utils.data.DataLoader(
             torch.utils.data.ConcatDataset([
                 temp,
                 temp2]),
             batch_size=2, shuffle=False,
             num_workers=2)

In [ ]:
it_temp = iter(temptotalloader)
next_temp, labels_temp = next(it_temp)
next_temp, labels_temp = Variable(next_temp), Variable(labels_temp)
print(next_temp)

In [ ]:
print(next_temp.shape)
for imageArr in next_temp:
    im = Image.fromarray((imageArr.detach().numpy() * 255).astype(np.uint8))
    im.save("temp/" + "temp.png")

## Attack

In [ ]:
def adv_attack(image, epsilon, data_grad, attack_method):
    assert attack_method in ['fgsm', 'stepll', 'jsma'] 
    
    # Collect the element-wise sign of the data gradient
    sign_data_grad = torch.sign(data_grad)
    
    # Create the perturbed image by adjusting each pixel of the input image
    if attack_method == 'fgsm':
        perturbed_image = image + epsilon*sign_data_grad
    else:
        perturbed_image = image - epsilon*sign_data_grad
        
    # Adding clipping to maintain [0,1] range
    perturbed_image = torch.clamp(perturbed_image, 0, 1)

    return perturbed_image

In [ ]:
def test(model, device, test_loader, criterion, attack_method, epsilon):
    assert attack_method in ['fgsm', 'stepll', 'jsma'] 
    
    # Accuracy counter
    correct = 0
    total = 0
    adv_examples = []

    # Loop over all examples in test set
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        
        # Set requires_grad attribute of tensor. Important for Attack
        data.requires_grad = True

        # Forward pass the data through the model
        output = model(data)
        print(output)
        init_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability

        if init_pred.item() != target.item(): # initially was incorrect --> no need to generate adversary
            continue
        
        if attack_method == 'fgsm':
            loss = criterion(output, target) # loss for ground-truth class
        else:
            ll = output.min(1, keepdim=True)[1][0]
            loss = criterion(output, ll)  # Loss for least-likely class
            
        # Back propogation
        model.zero_grad()
        loss.backward()

        # Collect data_grad
        data_grad = data.grad.data

        # Call Attack
        perturbed_data = adv_attack(data, epsilon, data_grad, attack_method)

        # Re-classify the perturbed image
        output = model(perturbed_data)

        # Check for success
        final_pred = output.max(1, keepdim=True)[1]
        if final_pred.item() == target.item():
            correct += 1 # still correct
            # Special case for saving 0 epsilon examples
        adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
        adv_examples.append((init_pred.item(), final_pred.item(), adv_ex))
        total += 1

    # Calculate final accuracy for this epsilon
    final_acc = correct/float(len(test_loader))
    print("Epsilon: {}\tTest Accuracy = {} / {} = {}".format(epsilon, correct, total, final_acc))

    # Return the accuracy and an adversarial example
    return final_acc, adv_examples

In [ ]:
accuracies = []
examples = []
epsilons = [0, .05, .1, .15, .2, .25, .3]
attack_method = 'fgsm'

for eps in epsilons:
    print(eps)
    acc, ex = test(model, device, testloader, criterion, attack_method, eps)
    accuracies.append(acc)
    examples.append(ex)

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(epsilons[:len(accuracies)], accuracies, "*-")
# plt.yticks(np.arange(0, 1.1, step=0.1))
# plt.xticks(np.arange(0, .35, step=0.05))
plt.title("Accuracy vs Epsilon")
plt.xlabel("Epsilon")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
accuracies = []
examples = []
epsilons = [0, .05, .1, .15, .2, .25, .3]
attack_method = 'stepll'

for eps in epsilons:
    acc, ex = test(model, device, testloader, criterion, attack_method, eps)
    accuracies.append(acc)
    examples.append(ex)

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(epsilons[:len(accuracies)], accuracies, "*-")
# plt.yticks(np.arange(0, 1.1, step=0.1))
# plt.xticks(np.arange(0, .35, step=0.05))
plt.title("Accuracy vs Epsilon")
plt.xlabel("Epsilon")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
import pickle

def load_cifar10_data(filename):
    with open('data/cifar-10-batches-py/'+ filename, 'rb') as file:
        batch = pickle.load(file, encoding='latin1')

    features = batch['data']
    labels = batch['labels']
    return features, labels

batch_1, labels_1 = load_cifar10_data('test_batch')
print(labels_1)